<a href="https://colab.research.google.com/github/rohilrao/Py4DS/blob/main/CUDA_Sheet4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [63]:
batch_size = 64
n_iters = 30000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = dsets.CIFAR10('./data', train=True, download=True, transform=transform)
 
test_dataset = dsets.CIFAR10('./data', train=False, download=True, transform=transform)
 
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
 
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [58]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
         
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=0)
        self.relu1 = nn.ReLU()
         
        # Max pool 1
        self.pool1 = nn.MaxPool2d(kernel_size=2)
      
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=0)
        self.relu2 = nn.ReLU()
         
        # Max pool 2
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        # Convolution 3
        self.cnn3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=0)
        self.relu3 = nn.ReLU()

        # Max pool 3
        self.pool3 = nn.MaxPool2d(kernel_size=2)
         
         
        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(128 * 2 * 2, 128)

        # FUlly connected 2       
        self.fc2 = nn.Linear(128, 10) 

        # Dropout
        self.Dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        # x =  torch.Size([100, 3, 32, 32])
        # Convolution 1

       
        out = self.cnn1(x) #torch.Size([100, 32, 30, 30])
        out = self.relu1(out) #torch.Size([100, 32, 30, 30 ])
         
        # Max pool 1
        out = self.pool1(out) #torch.Size([100, 32, 15, 15])

        # Dropout after firstlayer
        out = self.Dropout(out)

        # Convolution 2 
        out = self.cnn2(out) #torch.Size([100, 64, 13, 13])
        out = self.relu2(out) #torch.Size([100, 64, 13, 13])
     
        # Max pool 2 
        out = self.pool2(out) #torch.Size([100, 64, 6, 6])
    
        out = self.Dropout(out) 

        # Convolution 3
        out = self.cnn3(out) #torch.Size([100, 128, 4, 4])
        out = self.relu3(out) #torch.Size([100, 128, 4, 4])
     
        # Max pool 2 
        out = self.pool3(out) #torch.Size([100, 128, 2, 2])
    
        out = self.Dropout(out) 

        out = out.view(out.size(0), -1)   #torch.Size([100, 512])

        
 
        # Linear function (readout)
        out = self.fc1(out)
        out = self.Dropout(out)
        out = self.fc2(out)
         
        return out

In [59]:
model = CNNModel()
 
#######################
#  USE GPU FOR MODEL  #
#######################
 
if torch.cuda.is_available():
    model.cuda()

In [60]:
criterion = nn.CrossEntropyLoss()

In [61]:
learning_rate = 0.001
 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [62]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
         
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
        
        #print(images.shape)
        #print(labels.shape)

         
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
         
        # Forward pass to get output/logits
        outputs = model(images)
         
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
         
        # Getting gradients w.r.t. parameters
        loss.backward()
         
        # Updating parameters
        optimizer.step()
         
        iter += 1
         
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                if torch.cuda.is_available():
                    images = images.cuda()
                 
                # Forward pass only to get logits/output
                outputs = model(images)
                 
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                 
                # Total number of labels
                total += labels.size(0)
                 
                # Total correct predictions
                correct += (predicted.cpu() == labels.cpu()).sum().float()

             
            accuracy = 100. * correct / total
             
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 1.4304972887039185. Accuracy: 48.36000061035156
Iteration: 1000. Loss: 1.1633632183074951. Accuracy: 53.54999923706055
Iteration: 1500. Loss: 1.0777140855789185. Accuracy: 59.04999923706055
Iteration: 2000. Loss: 0.8869807720184326. Accuracy: 58.900001525878906
Iteration: 2500. Loss: 0.8831294178962708. Accuracy: 63.849998474121094
Iteration: 3000. Loss: 1.1146085262298584. Accuracy: 65.4000015258789
Iteration: 3500. Loss: 0.9344459176063538. Accuracy: 66.05999755859375
Iteration: 4000. Loss: 0.9461567997932434. Accuracy: 66.6500015258789
Iteration: 4500. Loss: 1.0535328388214111. Accuracy: 66.7699966430664
Iteration: 5000. Loss: 0.8243972063064575. Accuracy: 67.61000061035156
Iteration: 5500. Loss: 0.6755139231681824. Accuracy: 68.05000305175781
Iteration: 6000. Loss: 0.6112372875213623. Accuracy: 68.72000122070312
Iteration: 6500. Loss: 0.9604078531265259. Accuracy: 68.9800033569336
Iteration: 7000. Loss: 0.6965226531028748. Accuracy: 68.12000274658203
Iteration